In [ ]:
import json
import requests

import os
from pprint import pprint

import cltk

# run this only once to set up the corpus files for lemmatizing and such
# def setup_files():
    # from cltk.corpus.utils.importer import CorpusImporter

    # corpus_importer = CorpusImporter('latin')
    # corpus_importer.list_corpora
    # corpus_importer.import_corpus('latin_text_latin_library')
    # corpus_importer.import_corpus('latin_models_cltk')


from cltk.tokenize.word import WordTokenizer
from nltk.tokenize.punkt import PunktLanguageVars
from cltk.stop.latin.stops import STOPS_LIST


import os
from cltk.utils.file_operations import open_pickle
from cltk.lemmatize.latin.backoff import BackoffLatinLemmatizer

from cltk.stem.lemma import LemmaReplacer
from collections import Counter

#try this on the ars
with open('ars.json') as data_file:    
    data = json.load(data_file)

works = data['works']

line_list =[]
word_list = []

#set up tokenizer 
word_tokenizer = WordTokenizer('latin')

# set up fancy lemmatizer
# rel_path = os.path.join('~/cltk_data/latin/model/latin_models_cltk/lemmata/backoff')
# path = os.path.expanduser(rel_path)

# # Check for presence of latin_pos_lemmatized_sents
# file = 'latin_pos_lemmatized_sents.pickle'      

# latin_pos_lemmatized_sents_path = os.path.join(path, file)

# if os.path.isfile(latin_pos_lemmatized_sents_path):
#     latin_pos_lemmatized_sents = open_pickle(latin_pos_lemmatized_sents_path)
# else:
#     latin_pos_lemmatized_sents = []
#     print('The file %s is not available in cltk_data' % file)

# lemmatizer = BackoffLatinLemmatizer(latin_pos_lemmatized_sents)

#or just use default
default_lemmatizer = LemmaReplacer('latin')

# Remove punctuation with translate
punctuation ="\"#$%&\'()+,-/:;<=>@[\]^_`{|}~.?!«»—"
translator = str.maketrans({key: " " for key in punctuation})

word_list = []

# make list of lines
for work in works:
    books = work['books']
    for book in books:
        poems = book ['poems']
        for poem in poems:
            lines = poem['lines']
            for line in lines:
                try:
                    raw_line = line['text']
                    text = raw_line.translate(translator)
                    word_list = word_tokenizer.tokenize(text)
                    for word in word_list:
                        word = default_lemmatizer.lemmatize(word)
#                         print(word[0])
                        word_list.append(word[0])                         
                except:
                    pass
print('done')
lemma_text = ' '.join(word_list)
print(lemma_text)
# print(word_list)



# lemmas_counter = Counter(word_list)
# lemmas_mc = lemmas_counter.most_common(100)

# print (lemmas_mc)

#make sublist to test lemmatizer which is very slow
# small_list = line_list[1:3]
# print (small_list)

# for item in small_list:
#     for word in item:
#         word = lemmatizer.lemmatize(word)
#     print(item)

# print (small_list)
# lemmatizer.lemmatize('amabo')
# default_lemmatizer.lemmatize('amabo')
# default_lemmatizer.lemmatize('pedis')

# Refs:
# https://github.com/diyclassics/ll-experiments/blob/master/Exploring%20Diction%20and%20Topics%20in%20Latin%20Love%20Elegy.ipynb
# https://github.com/kynan/nbstripout

In [ ]:
print(word_list)